<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/PadhAI_Implementations/blob/main/PyTorch_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objective ###

*   Run pretrained models to fine tune last layers.
*   Experiment with VGG, Inception and ResNet




#### Using Pre-Trained models

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [3]:
import torch.optim as optim
import torch.nn as nn

In [6]:
device = "cuda:0" if torch.cuda.is_available() else"cpu"

In [21]:
device

'cuda:0'

##### Building transforms for VGG and ResNet #####

In [8]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

In [9]:
test_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

##### Building the transforms for Inception #####

In [ ]:
train_transform_inception = transforms.Compose([
    transforms.RandomResizedCrop(229),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

In [ ]:
test_transform_inception = transforms.Compose([
    transforms.RandomResizedCrop(229),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

##### Loading the dataset and data loaders #####

In [10]:
train_df = datasets.CIFAR10(root='./data',train = True, download = True, transform = train_transform)
test_df = datasets.CIFAR10(root='./data', train = False, download = True, transform = test_transform)

100%|██████████| 170498071/170498071 [00:13<00:00, 13065507.66it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [25]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size = 32, shuffle = True)

In [12]:
test_loader = torch.utils.data.DataLoader(test_df, batch_size = 16, shuffle = True)

##### VGG Model #####

###### Training from scratch ######

In [ ]:
vgg_model = models.vgg16_bn()

In [ ]:
print(vgg_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:
vgg_model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [ ]:
vgg_model.classifier[6] = nn.Linear(in_features=4096, out_features = 10)

In [ ]:
vgg_model.classifier[6]

Linear(in_features=4096, out_features=10, bias=True)

###### Using a pretrained model ######

1.  Load specific model and set pretrained = True
2.  Set all the requires_grad to False
3.  Modify the last layer of FFN according to the number of target classes.
4.  Build model and evaluate performance.     



In [36]:
vgg_model_pretrained = models.vgg16_bn(pretrained = True)

In [37]:
for param in vgg_model_pretrained.parameters():
    param.requires_grad = False

In [38]:
vgg_model_pretrained.classifier[6] = nn.Linear(in_features = 4096,out_features = 10)

In [39]:
for param in vgg_model_pretrained.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([10, 4096])
torch.Size([10])


In [40]:
vgg_model_pretrained = vgg_model_pretrained.to(device)

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model_pretrained.parameters(),lr=0.001)

In [42]:
def training_loop(dataloader, model, epochs):
    loss_per_epoch = []
    for epoch in range(epochs):
        for i, data in enumerate(dataloader,0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            #print(labels)

            output = model(images)
            #print(output) -> Displays probability for each class

            #predicted_class = torch.argmax(output,1)
            #print(predicted_class) -> Displays class with highest probability

            # Compute the cross entropy loss
            loss = loss_fn(output,labels)
            #print(loss)

            # Perform backpropagation
            loss.backward()

            # Update the weights and bias for the last layer
            optimizer.step()

            # Optimizing memory usage
            del images, labels, output
            torch.cuda.empty_cache()

            if i%50==0:
                print(f'Iteration {i}: Loss - {loss.item()}')

        loss_per_epoch.append(loss.item())
    return loss_per_epoch

In [43]:
loss = training_loop(train_loader,vgg_model_pretrained,1)

Iteration 0: Loss - 2.348021984100342
Iteration 50: Loss - 1.6975477933883667
Iteration 100: Loss - 1.5071234703063965
Iteration 150: Loss - 1.3905903100967407
Iteration 200: Loss - 1.862545371055603
Iteration 250: Loss - 1.056376338005066
Iteration 300: Loss - 1.5060253143310547
Iteration 350: Loss - 1.2138941287994385
Iteration 400: Loss - 1.4088847637176514
Iteration 450: Loss - 1.3011606931686401
Iteration 500: Loss - 1.0284245014190674
Iteration 550: Loss - 1.4073139429092407
Iteration 600: Loss - 1.2647435665130615
Iteration 650: Loss - 1.2926669120788574
Iteration 700: Loss - 1.7534215450286865
Iteration 750: Loss - 1.330113172531128
Iteration 800: Loss - 1.2406420707702637
Iteration 850: Loss - 1.2468408346176147
Iteration 900: Loss - 1.1193571090698242
Iteration 950: Loss - 1.396403431892395
Iteration 1000: Loss - 0.8315589427947998
Iteration 1050: Loss - 1.3147289752960205
Iteration 1100: Loss - 1.2102510929107666
Iteration 1150: Loss - 1.0698049068450928
Iteration 1200: Loss

### Experimental ###

In [ ]:
t = torch.tensor([[-0.6164,  0.0215, -0.6216, -0.5716,  0.4533,  0.1927, -0.5733, -0.2763,
         -0.2323, -0.3006],
        [ 0.1643,  0.4449, -0.5977, -0.1297,  0.0267,  0.1185,  0.1728, -0.0669,
         -0.1361, -0.1916],
        [ 0.1648,  0.1427,  0.1467,  0.0976,  0.0916, -0.3285,  0.0614,  0.4257,
         -0.4803, -0.2412],
        [ 0.0194,  0.2434, -0.5762,  0.2695, -0.1636, -0.0406,  0.2130,  0.3534,
          0.1299, -0.2418],
        [-0.0971, -0.0446, -0.4116, -0.1931,  0.4207,  0.3583,  0.1577,  0.0175,
         -0.1576,  0.0654],
        [-0.0044,  0.2831, -0.3403,  0.2046, -0.4859, -0.0732, -0.1655,  0.1770,
         -0.1208,  0.0334],
        [-0.5923,  0.1234, -0.1038, -0.1409, -0.1928,  0.7160, -0.0210, -0.1204,
         -0.1741, -0.4065],
        [-0.0934,  0.2186, -0.5412,  0.5249,  0.0304,  0.4698, -0.1730, -0.1892,
          0.0085,  0.4449]])

In [ ]:
torch.argmax(t,1)

tensor([4, 1, 7, 7, 4, 1, 5, 3])

In [ ]:
for i, data in enumerate(train_loader,0):
    image, label = data

    print(label)
    if i==1:
        break

tensor([2, 2, 3, 5, 3, 3, 3, 3])
tensor([7, 3, 1, 6, 8, 9, 3, 5])
